In [10]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import re

In [8]:
d = []
dirs = ['../data/processed/', '../data/processed_1', '../data/processed_2']
for dir in dirs:
    for fname in os.listdir(dir):
        print(fname)
        fname_ = os.path.join(dir, fname)
        if not fname_.endswith('.pkl') or 'b' in fname: continue 
        d.append(pickle.load(open(fname_, 'rb')))

998_b.pkl
968.pkl
1047.pkl
958_b.pkl
1176_b.pkl
1082.pkl
1081.pkl
879_b.pkl
1113_b.pkl
1159.pkl
1043.pkl
1181_b.pkl
1016_b.pkl
1160.pkl
877.pkl
1030_b.pkl
918_b.pkl
1100.pkl
924.pkl
1184.pkl
917.pkl
1082_b.pkl
1140.pkl
1102.pkl
1052_b.pkl
860.pkl
1059_b.pkl
941.pkl
847.pkl
1037_b.pkl
893_b.pkl
1056.pkl
970.pkl
1043_b.pkl
853_b.pkl
862_b.pkl
1039.pkl
857.pkl
1035_b.pkl
951_b.pkl
1104.pkl
950.pkl
1108_b.pkl
850_b.pkl
892_b.pkl
914_b.pkl
882.pkl
1011_b.pkl
899.pkl
941_b.pkl
1115_b.pkl
1187.pkl
1120_b.pkl
950_b.pkl
1055_b.pkl
979_b.pkl
1036_b.pkl
908.pkl
1033.pkl
1166_b.pkl
855.pkl
872.pkl
1176.pkl
933_b.pkl
885.pkl
1084.pkl
975.pkl
1144_b.pkl
1069.pkl
896.pkl
1094.pkl
908_b.pkl
1048_b.pkl
1125_b.pkl
921.pkl
1024_b.pkl
881_b.pkl
853.pkl
895_b.pkl
1035.pkl
1105_b.pkl
866_b.pkl
1061.pkl
960.pkl
1017_b.pkl
1127.pkl
1095_b.pkl
1029.pkl
952.pkl
991.pkl
1025_b.pkl
957.pkl
1015_b.pkl
1175.pkl
915_b.pkl
911.pkl
955_b.pkl
1107.pkl
1093.pkl
1086_b.pkl
1030.pkl
1172.pkl
1189.pkl
1165.pkl
1018_b.pkl
1

In [11]:
bad_rows = []
dfs = []
for d_ in d:
    parsed_data = {}
    for item in d_:
        id, string_data = item
        split_data = [s.strip().replace("[", "").replace("]", "").replace(",", "") for s in string_data.split('],')]
        parsed_data[id] = split_data
    data = []
    rows = []
    keys = ['1(neg)', '2', '3', '4', '5(pos)']
    categories = ["Mostly related:", "Somewhat related:", "Unrelated:"]

    columns = ['id'] + [f's_{i}' for i in range(5)] + [f'a_{i}' for i in range(2)] + [f'r_{i}' for i in range(3)]
    for k, v in parsed_data.items():
        try:
            ## parse the sentiment
            try:
                v_ = re.sub(r'^[^0-9]*', '', v[0]) 
                kv_pairs = v_.split()
                kv_dict = {kv_pairs[i]: float(kv_pairs[i+1].replace(':', '')) for i in range(0, len(kv_pairs), 2)}
                
                # Extract the values for the specified keys, if they exist
                sentiment_row = [kv_dict.get(key + ':', 0) for key in keys]
            except Exception as e:
                print(e)
                sentiment_row = [0.2] * 5
    
            try:
                ## No clear advantage
                if 'NA' in v[1]:
                    advantage = [0.5, 0.5]
                else:
                    advantage = list(map(float, re.sub(r'[^0-9.]', ' ', v[1]).split()))
            except Exception as e:
                advantage = [0.5, 0.5]
    
            try:
                if len(v) < 3: 
                    values = [0.33, 0.33, 0.33]
                else:
                    # Define the categories in the required order
                    
                    # Initialize a list to hold the cumulative values
                    values = [0, 0, 0]
                    # Split the line by space to get label-value pairs
                    pairs = v[2].split()
                    
                    for category in categories:
                        if category in v[2]:
                            # Extract value after the category label
                            value_str = v[2].split(category)[1].split()[0]
                            if value_str == 'NA':
                                value = 0
                            else:
                                value = float(value_str)
                            index = categories.index(category)
                            values[index] += value
            except Exception as e:
                values = [0.33, 0.33, 0.33]
                
            row_values = [k] + sentiment_row + advantage + values
            if len(row_values) != 11:
                bad_rows.append(row_values)
            else:
                rows.append(row_values)
        

        except Exception as e:
            bad_rows.append()
            
    df_ = pd.DataFrame(rows, columns = columns)

    dfs.append(df_)

could not convert string to float: 'Relation'
could not convert string to float: 'related'
could not convert string to float: 'Disadvantage'
could not convert string to float: 'Positive'
could not convert string to float: '0.8(pos)'
could not convert string to float: 'Disadvantage'
could not convert string to float: 'related'
could not convert string to float: '0.8(pos)'
could not convert string to float: 'related'
could not convert string to float: 'related'
could not convert string to float: 'Advantage'
could not convert string to float: "related'"
could not convert string to float: 'NA'
could not convert string to float: 'related'
could not convert string to float: 'related'
could not convert string to float: 'Relation'
could not convert string to float: 'related'
could not convert string to float: ''
could not convert string to float: 'related'
could not convert string to float: ''
could not convert string to float: 'related'
list index out of range
could not convert string to floa

In [12]:
df = pd.concat(dfs)
# Remove duplicate indexes
df = df.groupby('id').first().reset_index()

In [14]:
df.to_csv('../data/cleaned/sentiment2.csv')